In [14]:
import csv
from collections import defaultdict

data = defaultdict(list)

with open('Description.csv', 'r', encoding = 'utf-8') as infh:
    reader = csv.reader(infh)
    next(reader, None)  # skip the header

    for col1, col2 in reader:
        data[col1].append(col2)



In [15]:
classes = list(data.keys())

In [16]:
classes[1]

'Cats'

In [17]:
import json

dat =json.dumps(data)
with open('desc.json', 'w') as outfile:
    json.dump(data, outfile)

In [18]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import string
import unicodedata
import sys

In [19]:
import unicodedata
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

In [20]:
def remove_punctuation(text):
    return text.translate(tbl)

In [21]:
stemmer = LancasterStemmer()
# variable to hold the Json data read from the file
data = None

# read the json file and load the training data
with open('desc.json') as json_data:
    data = json.load(json_data)
    #print(data)

In [22]:
categories = list(data.keys())
words = []
# a list of tuples with words in the sentence and category name
docs = []

In [23]:
for each_category in data.keys():
    for each_sentence in data[each_category]:
        # remove any punctuation from the sentence
        each_sentence = remove_punctuation(each_sentence)
        #print(each_sentence)
        # extract words from each sentence and append to the word list
        w = nltk.word_tokenize(each_sentence)
        #print("tokenized words: ", w)
        words.extend(w)
        docs.append((w, each_category))

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

In [24]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(categories)


for doc in docs:
    # initialize our bag of words(bow) for each document in the list
    bow = []
    # list of tokenized words for the pattern
    token_words = doc[0]
    # stem each word
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # create our bag of words array
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)

    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1

    # our training set will contain a the bag of words model and the output row that tells
    # which catefory that bow belongs to.
    training.append([bow, output_row])
# shuffle our features and turn into np.array as tensorflow  takes in numpy array
random.shuffle(training)
training = np.array(training)

# trainX contains the Bag of words and train_y contains the label/ category
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

In [ ]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)
model.save('descmodel.tflearn')


# let's test the mdodel for a few sentences:
# the first two sentences are used for training, and the last two sentences are not present in the training data

# a method that takes in a sentence and list of all words
# and returns the data in a form the can be fed to tensorflow


def get_tf_record(sentence):
    global words
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    # bag of words
    bow = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bow[i] = 1

    return(np.array(bow))

Training Step: 8976  | total loss: 0.36607 | time: 0.044s
| Adam | epoch: 449 | loss: 0.36607 - acc: 0.8505 -- iter: 128/159


In [36]:
print(categories[np.argmax(model.predict([get_tf_record("this animal uses vocalizations to communicate")]))])

Monkeys


In [95]:
p = model.predict([get_tf_record("hunts in groups")])
print(categories[np.argmax(p)])

Elephants


In [96]:
d = p[0].tolist()

In [97]:
for i in d:
    if i >= (max(d)-0.2) and i <= (max(d)+0.2):
        print(categories[d.index(i)])

Elephants
Lions
Tigers
Monkeys
